In [1]:
import pm4py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
log = pm4py.read_xes('../data/BPI Challenge 2017.xes/BPI Challenge 2017.xes', return_legacy_log_object=True)

c:\Users\Henri\OneDrive - Universität zu Köln\Dokumente\TUM\3. Semester\BPTM Seminar\.venv\Lib\site-packages\pm4py\utils.py:992: UserWarning: Install the optional requirement `rustxes` to import/export files faster.
  warnings.warn("Install the optional requirement `rustxes` to import/export files faster.")
c:\Users\Henri\OneDrive - Universität zu Köln\Dokumente\TUM\3. Semester\BPTM Seminar\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 31509/31509 [00:39<00:00, 793.51it/s] 


In [6]:
log[0]

{'attributes': {'LoanGoal': 'Existing loan takeover', 'ApplicationType': 'New credit', 'concept:name': 'Application_652823628', 'RequestedAmount': 20000.0}, 'events': [{'Action': 'Created', 'org:resource': 'User_1', 'concept:name': 'A_Create Application', 'EventOrigin': 'Application', 'EventID': 'Application_652823628', 'lifecycle:transition': 'complete', 'time:timestamp': datetime.datetime(2016, 1, 1, 9, 51, 15, 304000, tzinfo=datetime.timezone.utc)}, '..', {'Action': 'Deleted', 'org:resource': 'User_115', 'concept:name': 'W_Validate application', 'EventOrigin': 'Workflow', 'EventID': 'Workitem_2067229931', 'lifecycle:transition': 'ate_abort', 'time:timestamp': datetime.datetime(2016, 1, 14, 15, 49, 11, 423000, tzinfo=datetime.timezone.utc)}]}

In [7]:
log_start = pm4py.get_start_activities(log)
log_start

{'A_Create Application': 31509}

## Analyze characteristics of start activities

In [8]:
filtered_log = pm4py.filter_start_activities(log, ["A_Create Application"])
filtered_log

[{'attributes': {'LoanGoal': 'Existing loan takeover', 'ApplicationType': 'New credit', 'concept:name': 'Application_652823628', 'RequestedAmount': 20000.0}, 'events': [{'Action': 'Created', 'org:resource': 'User_1', 'concept:name': 'A_Create Application', 'EventOrigin': 'Application', 'EventID': 'Application_652823628', 'lifecycle:transition': 'complete', 'time:timestamp': datetime.datetime(2016, 1, 1, 9, 51, 15, 304000, tzinfo=datetime.timezone.utc)}, '..', {'Action': 'Deleted', 'org:resource': 'User_115', 'concept:name': 'W_Validate application', 'EventOrigin': 'Workflow', 'EventID': 'Workitem_2067229931', 'lifecycle:transition': 'ate_abort', 'time:timestamp': datetime.datetime(2016, 1, 14, 15, 49, 11, 423000, tzinfo=datetime.timezone.utc)}]}, '....', {'attributes': {'LoanGoal': 'Home improvement', 'ApplicationType': 'New credit', 'concept:name': 'Application_1350494635', 'RequestedAmount': 20000.0}, 'events': [{'Action': 'Created', 'org:resource': 'User_1', 'concept:name': 'A_Creat

#### Inital log exploration

In [22]:
k = 3000
filtered_variants = pm4py.filter_variants_top_k(log, k)

In [23]:
len(filtered_variants)

18579

In [25]:
log[0].attributes

{'LoanGoal': 'Existing loan takeover',
 'ApplicationType': 'New credit',
 'concept:name': 'Application_652823628',
 'RequestedAmount': 20000.0}

In [28]:
loan_amounts = [event.attributes['RequestedAmount'] for event in log]

In [31]:
print(np.mean(loan_amounts))
print(np.std(loan_amounts))

16233.74398933638
15422.001569645281


In [ ]:
# retain only cases with a final decision by the bank

allowed_events = ["A_Pending", "A_Cancelled", "A_Denied"]

filtered_log = pm4py.filter_event_attribute_values(
    log,
    attribute_key="concept:name",
    values=allowed_events,
    level="case",
    retain=True
)


In [39]:
len(filtered_log)

31411

#### Initial descriptive analysis

In [ ]:
count_cases = len(log)
case_lengths = [len(trace) for trace in log]
total_events_sum = sum(case_lengths)
case_length_mean = np.mean(case_lengths)
case_length_std = np.std(case_lengths)
variants = pm4py.get_variants(log)
count_variants = len(variants.items())
case_labels = set(
    trace.attributes.get("concept:name") or 
    trace.attributes.get("case:concept:name")
    for trace in log
)
event_labels = set(
    event["concept:name"]
    for trace in log
    for event in trace
)

# get durations of events
durations = []

for trace in log:
    # extract timestamps from events
    timestamps = [event["time:timestamp"] for event in trace]

    timestamps = sorted(timestamps)

    start = timestamps[0]
    end   = timestamps[-1]

    duration = end - start
    durations.append(duration)

durs = pd.to_timedelta(durations)

mean_duration = durs.mean()
std_duration  = durs.std()

loan_amounts = [event.attributes['RequestedAmount'] for event in log]

print(f"No. of cases: {count_cases}")
print(f"No. of events: {total_events_sum}")
print(f"No- of variants: {count_variants}")
print(F"No. of case labels: {len(case_labels)}")
print("Unique event labels:", len(event_labels))
print(F"Mean and std. dev. of case length: Mean: {case_length_mean} Std: {case_length_std}")
print(f"Mean and std. dev. of case duration: Mean: {mean_duration} Std: {std_duration}")
print(f"Mean and std. dev. of loan amount Mean: {np.mean(loan_amounts)} Std: {np.std(loan_amounts)}")

No. of cases: 31509
No. of events: 1202267
No- of variants: 15930
No. of case labels: 31509
Unique event labels: 26
Mean and std. dev. of case length: Mean: 38.15630454790695 Std: 16.715042807894445
Mean and std. dev. of case duration: Mean: 21 days 21:35:25.923069948 Std: 13 days 04:03:41.690875763
Mean and std. dev. of loan amount Mean: 16233.74398933638 Std: 15422.001569645281
